In [1]:
import requests
import pandas as pd
from config import app_id, api_key

In [2]:
df = pd.read_csv('austin_locations_all.csv')
df.head()

,Unnamed: 0,name,brand_id,fs_id,address,address2,city,state,country,zip,phone,website,guide,id,lat,lng,created_at,updated_at,distance_km
0,80,Subway,513fbc1283aa2dc80c000005,NaN,11215 S INTERSTATE 35 STE 128,NaN,Austin,TX,US,78747,1.512283e+10,http://www.subway.com,"<p><b id=""docs-internal-guid-6dd52ca7-83bb-38f...",837595,30.136999,-97.795998,2017-06-26T21:59:11.000Z,2017-08-20T14:02:38.000Z,15.863686
1,216,Subway,513fbc1283aa2dc80c000005,NaN,6422 HIGHWAY 183 S,NaN,Austin,TX,US,78744,1.512243e+10,http://www.subway.com,"<p><b id=""docs-internal-guid-6dd52ca7-83bb-38f...",796835,30.165001,-97.693001,2017-06-26T21:54:34.000Z,2017-08-20T14:07:28.000Z,12.187233
2,217,Auntie Anne's,513fbc1283aa2dc80c00013e,4cadcc8bd1f8b60c8cb771c6,3600 Presidential Blvd Ste 204,NaN,Austin,TX,US,78719,NaN,http://www.auntieannes.com,NaN,2492829,30.202162,-97.666260,2018-09-21T14:06:53.000Z,2019-01-05T14:56:54.000Z,12.271612
3,218,Thundercloud Subs,513fbc1283aa2dc80c000b53,NaN,3600 PRESIDENTIAL BLVD #213,NaN,Austin,TX,US,78719,1.512530e+10,http://thundercloud.com/,NaN,790078,30.202148,-97.666832,2017-06-26T21:53:40.000Z,2017-08-20T14:06:12.000Z,12.309972
4,219,Schlotzsky's,513fbc1283aa2dc80c00013a,NaN,3600 PRESIDENTIAL BLVD,NaN,Austin,TX,US,78719,1.512530e+10,http://www.schlotzskys.com,NaN,1869663,30.202635,-97.667381,2017-07-12T17:55:52.000Z,2017-08-20T14:06:32.000Z,12.385516


In [3]:
brands = df['brand_id'].unique()
len(brands)

121

In [4]:
menu_url = 'https://trackapi.nutritionix.com/v2/search/instant'
headers = {'x-app-id': app_id, 'x-app-key':api_key}

In [5]:
branded_item_list = []
for brand in brands:
    params = {'query': 'burrito', 'brand_ids': brand, 'branded': True, 'branded_type': 1, 'detailed': True}
    response_menu_items = requests.get(menu_url, headers=headers, params=params).json()
    if response_menu_items['branded'] != []:
        branded_item_list.append(response_menu_items['branded'])

In [6]:
item_brand = [branded_item_list[i][j]['brand_name'] for i in range(len(branded_item_list)) for j in range(len(branded_item_list[i]))]
item_name = [branded_item_list[i][j]['food_name'] for i in range(len(branded_item_list)) for j in range(len(branded_item_list[i]))]
nutrient_dict = [branded_item_list[i][j]['full_nutrients'] for i in range(len(branded_item_list)) for j in range(len(branded_item_list[i]))]
calories = [branded_item_list[i][j]['nf_calories'] for i in range(len(branded_item_list)) for j in range(len(branded_item_list[i]))]

In [7]:
items_df = pd.DataFrame({'Restaurant': item_brand,
                        'Food_Name': item_name,
                        'Nutrients': nutrient_dict,
                        'Calories': calories})
items_df.tail(10)

,Restaurant,Food_Name,Nutrients,Calories
141,La Salsa Fresh Mexican Grill,"3 Pepper Burrito, Carnitas","[{'value': 42, 'attr_id': 203}, {'value': 35, ...",780
142,La Salsa Fresh Mexican Grill,"3 Pepper Burrito, Shrimp","[{'value': 35, 'attr_id': 203}, {'value': 34, ...",940
143,La Salsa Fresh Mexican Grill,Los Cabos Shrimp Burrito,"[{'value': 31, 'attr_id': 203}, {'value': 34, ...",720
144,La Salsa Fresh Mexican Grill,"Original Gourmet Burrito, Chicken","[{'value': 37, 'attr_id': 203}, {'value': 41, ...",870
145,La Salsa Fresh Mexican Grill,"Overstuffed Grilled Burrito, Chicken","[{'value': 73, 'attr_id': 203}, {'value': 59, ...",1260
146,La Salsa Fresh Mexican Grill,"Overstuffed Grilled Burrito, Steak","[{'value': 66, 'attr_id': 203}, {'value': 66, ...",1290
147,La Salsa Fresh Mexican Grill,Side of Chips for Burritos,"[{'value': 3, 'attr_id': 203}, {'value': 10, '...",200
148,La Salsa Fresh Mexican Grill,California Burrito with Pinto Beans,"[{'value': 27, 'attr_id': 203}, {'value': 29, ...",690
149,La Salsa Fresh Mexican Grill,California Burrito with Black Beans,"[{'value': 28, 'attr_id': 203}, {'value': 29, ...",710
150,La Salsa Fresh Mexican Grill,Bean & Cheese Burrito with Black Beans,"[{'value': 48, 'attr_id': 203}, {'value': 48, ...",1100


In [13]:
items_df.to_csv('burritos.csv')